In [1]:
!pip install skrub -q

## Load Libraries

In [29]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler,Normalizer,OneHotEncoder
from skrub import SimilarityEncoder
import plotly.express as px
from sklearn.decomposition import PCA,TruncatedSVD

## Preparacion del dataset

In [30]:
dataframe = pd.read_csv('/content/Producción_Cultivos_ProductosGanaderia_S_Todos_los_Datos_NOFLAG (1).csv',encoding='latin-1')
non_important_features = ['Código del área', 'Código del área (M49)',
                          'Código del producto', 'Código del producto (CPC)',
                          'Código del elemento']
dataframe.drop(non_important_features,axis=1,inplace=True)
categorical_features = ['Área', 'Elemento', 'Producto', 'Unidad']
numerical_features = (x for x in dataframe.columns if x not in categorical_features)
numerical_data = dataframe[numerical_features].replace({np.nan : 0})
categorical_data = dataframe[categorical_features].replace({np.nan : 'unknown'})

## Creacion de la clase

In [49]:
class Pipeline:
    def load_data(self, numerical_df, categorical_df):
        self.num = numerical_df
        self.cat = categorical_df

    def preprocess_data(self, cat_encoder, num_scaler):
        self.cat_encoder = cat_encoder
        self.num_scaler = num_scaler

        if num_scaler == 'Standar':
            scaler = StandardScaler()
            self.numerical_data_preprocessed = scaler.fit_transform(self.num)
        elif num_scaler == 'MinMax':
            scaler = MinMaxScaler()
            self.numerical_data_preprocessed = scaler.fit_transform(self.num)
        elif num_scaler == 'Robust':
            scaler = RobustScaler()
            self.numerical_data_preprocessed = scaler.fit_transform(self.num)
        else:
            scaler = Normalizer()
            self.numerical_data_preprocessed = scaler.fit_transform(self.num)

        if cat_encoder == 'OneHot':
            encoder = OneHotEncoder()
            encoder.fit(self.cat)
            self.categorical_data_preprocessed = encoder.fit_transform(self.cat)
        elif cat_encoder == 'Similarity':
            encoder = SimilarityEncoder()
            encoder.fit(self.cat)
            self.categorical_data_preprocessed = encoder.fit_transform(self.cat)

    def visualize_data(self,data_graph_type):
        self.data_graph_type = data_graph_type
        if data_graph_type == 'Numerical':
          pca = PCA(n_components=2)
          numerical_transformed_data = pca.fit_transform(self.numerical_data_preprocessed)
          numerical_transformed_df = pd.DataFrame(numerical_transformed_data, columns=[f'Componente{i}' for i in range(1, pca.n_components_+1)])
          fig = px.scatter(numerical_transformed_df, x='Componente1', y='Componente2')
          fig.show()
        if data_graph_type == 'Categorical':
          pca = TruncatedSVD(n_components=2)
          categorical_transformed_data = pca.fit_transform(self.categorical_data_preprocessed)
          num_components = 2
          categorical_transformed_df = pd.DataFrame(categorical_transformed_data, columns=[f'Componente{i}' for i in range(1, num_components+1)])
          fig = px.scatter(categorical_transformed_df, x='Componente1', y='Componente2')
          fig.show()

In [50]:
pipeline_instance = Pipeline()

In [54]:
pipeline_instance.load_data(numerical_data,categorical_data)

In [57]:
pipeline_instance.preprocess_data('Similarity','Standar')

In [58]:
pipeline_instance.visualize_data('Categorical')